# GloVe

Constraint: Only use our code (not other code....)

1. I guess you already try a bigger corpus
2. I guess you already try window size 2
3. I guess you already have skipgram, skipgram(neg), cbow, glove

Do this:
1. Compare them based on syntactic accuracy and semantic accuracy, similar to how is done in https://nlp.stanford.edu/pubs/glove.pdf (see Table 2) - NO NEED to try 1000 or 300 embed size.....I just want you to learn how to do experiment.....
2. Try to find a correlation with just ONE similarity dataset (which humans judge how similar is two words.....)

Point criteria:
0:  Not done
1: ok
2: with comments / explanation / figures just like how Chaky explain thing.....

Some suggestions for the assignment report:
1. Write a brief conclusion on your findings at the end of the report.
2. Clear long outputs in the notebook before submission.

In [3]:
#importing libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import spacy
import nltk

## 1. Load Data

In [6]:
#using real corpus from nltk
from nltk.corpus import brown
nltk.download('brown')
corpus = nltk.corpus.brown.sents(categories=['reviews'])

[nltk_data] Downloading package brown to C:\Users\Sirikit
[nltk_data]     Joshi\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [7]:
#let's check the size of the corpus
print(len(corpus))

1751


In [13]:
#1. Tokenize
corpus_tokenized = [[word.lower() for word in sent] for sent in corpus]
print(corpus_tokenized[:5])

[['it', 'is', 'not', 'news', 'that', 'nathan', 'milstein', 'is', 'a', 'wizard', 'of', 'the', 'violin', '.'], ['certainly', 'not', 'in', 'orchestra', 'hall', 'where', 'he', 'has', 'played', 'countless', 'recitals', ',', 'and', 'where', 'thursday', 'night', 'he', 'celebrated', 'his', '20th', 'season', 'with', 'the', 'chicago', 'symphony', 'orchestra', ',', 'playing', 'the', 'brahms', 'concerto', 'with', 'his', 'own', 'slashing', ',', 'demon-ridden', 'cadenza', 'melting', 'into', 'the', 'high', ',', 'pale', ',', 'pure', 'and', 'lovely', 'song', 'with', 'which', 'a', 'violinist', 'unlocks', 'the', 'heart', 'of', 'the', 'music', ',', 'or', 'forever', 'finds', 'it', 'closed', '.'], ['there', 'was', 'about', 'that', 'song', 'something', 'incandescent', ',', 'for', 'this', 'brahms', 'was', 'milstein', 'at', 'white', 'heat', '.'], ['not', 'the', 'noblest', 'performance', 'we', 'have', 'heard', 'him', 'play', ',', 'or', 'the', 'most', 'spacious', ',', 'or', 'even', 'the', 'most', 'eloquent', '.'

In [14]:
#2. numericalize

#2.1 get all the unique words
#we want to flatten this (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs  = list(set(flatten(corpus_tokenized)))  #vocabs is a term defining all unique words your system know
print(vocabs[0:5])

['smoothness', 'brahms', 'nor', 'bang-sashes', 'adaptation']


In [19]:
#2.2 assign id to all these vocabs
word2index = {v: idx+1 for idx, v in enumerate(vocabs)}

#add <UNK>, which is a very normal token exists in the world
vocabs.append('<UNK>') #chaky, can it be ##UNK, or UNKKKKKK, or anything

word2index['<UNK>'] = 0 
# word2index

In [21]:
#create index2word dictionary
index2word = {v:k for k, v in word2index.items()}
# index2word